In [2]:
import pandas as pd
import re
import os
import string
import matplotlib.pyplot as plt
from os import listdir
import numpy as np
from sklearn.model_selection import train_test_split
# from keras.preprocessing import sequence
# from keras.models import Sequential
# from keras.layers import Dense
# from keras.layers import LSTM

# from keras.optimizers import Adam
# from keras.models import load_model
# from keras.callbacks import ModelCheckpoint

from matplotlib import rc
from matplotlib.ticker import MaxNLocator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split as tts      
from sklearn.preprocessing import StandardScaler as SS
from sklearn.metrics import confusion_matrix as CM 
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score as AS
from sklearn.linear_model import LinearRegression as LR
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.tree import DecisionTreeClassifier as dtc
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn import tree 
from multiprocessing import cpu_count

In [3]:
directory = './data/'
csv_files = [file for file in os.listdir(directory) if file.endswith('.csv')]
frames = {}
for csv_file in csv_files:
    file_path = os.path.join(directory, csv_file)
    df_name = os.path.splitext(csv_file)[0]  # Use file name as DataFrame name (without extension)
    df = pd.read_csv(file_path)
    frames[df_name] = df

for title, frame in frames.items():
    translation_table = str.maketrans('', '', string.digits)
    sitename = title.split("_")[0].translate(translation_table)
    frame['Site'] = sitename

In [4]:
import shutil

def delete_files_in_folder(folder_path):
    # Delete all files in the folder (but not subdirectories)
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        try:
            if os.path.isfile(file_path):  # Check if it's a file (not a directory)
                os.remove(file_path)       # Delete the file
                print(f"Deleted: {file_path}")
        except Exception as e:
            print(f"Error deleting {file_path}: {e}")

# Specify the folder path
folder_path = './data2'

# Call the function to delete files in the folder
delete_files_in_folder(folder_path)

Deleted: ./data2/bing_079.csv
Deleted: ./data2/bing_051.csv
Deleted: ./data2/bing_045.csv
Deleted: ./data2/amazon_032.csv
Deleted: ./data2/amazon_026.csv
Deleted: ./data2/bing_092.csv
Deleted: ./data2/discord_140.csv
Deleted: ./data2/bing_086.csv
Deleted: ./data2/ryrod_107.csv
Deleted: ./data2/cnn_103.csv
Deleted: ./data2/youtube_021.csv
Deleted: ./data2/ryrod_113.csv
Deleted: ./data2/cnn_117.csv
Deleted: ./data2/youtube_035.csv
Deleted: ./data2/ugm_066.csv
Deleted: ./data2/youtube_009.csv
Deleted: ./data2/zoom_003.csv
Deleted: ./data2/zoom_017.csv
Deleted: ./data2/Netflix_003.csv
Deleted: ./data2/wpbeginner_046.csv
Deleted: ./data2/Netflix_017.csv
Deleted: ./data2/cnn_077.csv
Deleted: ./data2/ryrod_073.csv
Deleted: ./data2/minecraft_039.csv
Deleted: ./data2/cnn_063.csv
Deleted: ./data2/ryrod_067.csv
Deleted: ./data2/hulu_060.csv
Deleted: ./data2/hulu_048.csv
Deleted: ./data2/minecraft_011.csv
Deleted: ./data2/minecraft_005.csv
Deleted: ./data2/amazonvideo_032.csv
Deleted: ./data2/amaz

In [5]:
def remove_numbers(input_string):
    if type(input_string) == float:
        return input_string
    if 'Application Data' in input_string:
        return 'Application Data'
    if 'Payload' in input_string:
        return 'Payload'
    if 'Handshake' in input_string:
        return 'Handshake'
    translation_table = str.maketrans('', '', '0123456789')
    return input_string.translate(translation_table)

def info_dic_map(input_string):
    return info_dic[input_string]

def apply_target(df):
    if df['Site'].iloc[0] in ('amazon','chatgpt' ,'bing','discord' , 'googledrive' , 'wmregistration' , 'quizlet'):
        df['Target'] = 'sud'
        return df
    elif df['Site'].iloc[0] in ('discordstream' ,'fortnite' , 'minecraft', 'zoom'):
        df['Target'] = 'cud'
        return df
    elif df['Site'].iloc[0] in ('youtube' ,'hulu', 'Netflix', 'spotify', 'amazonvideo'):
        df['Target'] = 'cd'
        return df
    elif df['Site'].iloc[0] in ('cnn', 'Wikipedia', 'ryrod', 'ugm'):
        df['Target'] = 'sd'
    else:
        print(df['Site'].iloc[0])
        print("AAAA")
    return df
def extract_values(text):
    if type(text) == (float or int):
        return text
    # Use regex pattern to find values enclosed within square brackets
    pattern = r'\[([^\]]+)\]'  # Matches anything inside square brackets
    matches = re.findall(pattern, text)
    if not matches:
        return text
    return matches[0]

In [6]:
site_dic = {}
info_dic = {}
j = 0
user_li = ['100.69.171.196', '192.168.0.92', '192.168.0.129', '192.168.0.1', '100.86.171.196', '192.168.0.104',
           '192.168.0.200', '100.86.7.137', '2600:8805:3e22:4100:68dd:8e31:1a2:1f62']
site_li = ['18.160.17.214', '172.66.40.147', '104.234.169.167', '162.159.128.232', '70.186.28.136',
           '169.254.243.145', '74.125.155.72', '206.247.128.13', '144.195.6.47', '204.79.197.200',
           '104.18.37.228', '66.22.231.191', '66.22.196.159', '35.214.213.22', '23.48.104.108',
           '23.48.0.110', '13.32.208.51', '3.14.34.81', '3.144.65.185', '172.253.122.139', '209.222.115.47',
           '104.16.133.27','23.48.104.112', '20.36.181.22', '104.18.10.41', '172.66.43.109',
           '13.84.36.2', '208.80.154.224', '2607:f8b0:4004:f::a']
for title, frame in frames.items():
    frame['Time Delta'] = frame['Time'].diff().fillna(0)
    #1 = user
    #0 = site
    for i in user_li:
      frame['Source'] = frame['Source'].str.replace(i, '1')
      frame['Destination'] = frame['Destination'].str.replace(i, '1')
    for i in site_li:
      frame['Source'] = frame['Source'].str.replace(i, '0')
      frame['Destination'] = frame['Destination'].str.replace(i, '0')

    frame['Info'] = frame['Info'].apply(extract_values)

    for i in frame['Destination'].unique():
        if (i != '1' and i!='0'):
            frame['Source'] = frame['Source'].str.replace(i, '0')
            frame['Destination'] = frame['Destination'].str.replace(i, '0')
    for i in frame['Source'].unique():
        if (i != '1' and i!='0'):
            frame['Source'] = frame['Source'].str.replace(i, '0')
            frame['Destination'] = frame['Destination'].str.replace(i, '0')

    frame = apply_target(frame)

    protocol_map = {'UDP':0, 'TCP':1, 'TLSv1.2':2, 'TLSv1.3':3, 'RTCP': 4, 'QUIC': 5, 'SSDP':6, 'R-GOOSE':7, 'IGMPv2':8, 'ARP':9, 'WireGuard':10}
    frame['Protocol'] = frame['Protocol'].replace(protocol_map)

    translation_table = str.maketrans('', '', string.digits)
    sitename = title.split("_")[0].translate(translation_table)
    frame['Info'] = frame['Info'].apply(remove_numbers)
    for value in frame['Info'].unique():
        if value not in info_dic:
            j+=1
            info_dic[value] = j

    frame['Info'] = frame['Info'].apply(info_dic_map)

    if sitename not in site_dic:
        site_dic[sitename] = 0
    rows_per = 50
    total_dataframes = (len(frame)) // rows_per
    for i in range(total_dataframes):
        site_dic[sitename] += 1
        file_path = './data2/' + sitename + '_' + str("{:03d}".format(site_dic[sitename]) + '.csv')
        start_index = i * rows_per
        end_index = (i + 1) * rows_per
        smaller_df = frame.iloc[start_index:end_index]
        smaller_df.to_csv(file_path, index=False)

wpbeginner
AAAA
wpbeginner
AAAA
wpbeginner
AAAA
wpbeginner
AAAA
wpbeginner
AAAA


In [40]:
target_map = {'discord':0, 'bing':1, 'amazon':2, 'minecraft':3}

In [65]:
# Path to the folder containing CSV files
folder_path = './data2/'

# List to store all dataframes from CSV files
all_dataframes = []
test_dfs = []
data = pd.DataFrame()
# Iterate over each file in the folder
i = 0
#y_train = pd.DataFrame(columns=['Series', 'Website'])
for file_name in os.listdir(folder_path):
    if (file_name.startswith('discord_') or file_name.startswith('bing_') or file_name.startswith('amazon_') 
        or file_name.startswith('minecraft_'))and(file_name.endswith('.csv')):
        file_path = os.path.join(folder_path, file_name)
        # Read the CSV file into a dataframe
        df = pd.read_csv(file_path)
        df = df.drop(columns=['Source','Destination','Time','Target','No.'])
        df['Flow'] = i
        #df = df._append({'Series': i, 'Website': target_map[df.iloc[0]['Site']]}, ignore_index=True)
        i+=1
        all_dataframes.append(df)
        # Append the dataframe to the list
        #data = pd.concat(df,ignore_index = True)
#df = df.replace(target_map)
# Concatenate all dataframes into a single dataframe
combined_df = pd.concat(all_dataframes, ignore_index=True)
data = combined_df
# Path to the new combined CSV file
output_csv_path = './data3/data.csv'

# Write the combined dataframe to a new CSV file
#combined_df.to_csv(output_csv_path, index=False)
#y_train.to_csv('./data3/y_train.csv', index=False)
data.to_csv(output_csv_path, index = False)
print(f"Combined CSV file saved to: {output_csv_path}")
data = data.replace(target_map)
data



Combined CSV file saved to: ./data3/data.csv


,Protocol,Length,Info,Site,Time Delta,Flow
0,1,1514,3,1,0.000001,0
1,1,1514,3,1,0.000000,0
2,2,1354,2,1,0.000001,0
3,2,92,2,1,0.000001,0
4,1,56,3,1,0.000000,0
...,...,...,...,...,...,...
14945,1,54,3,1,0.000054,298
14946,1,54,16,1,0.000090,298
14947,2,1268,2,1,0.001571,298
14948,1,1514,3,1,0.000002,298


In [79]:
#TTS 
x = data.drop(columns = 'Site')
y = data['Site']
X = x.iloc[:,0:5]


X_train, X_test, y_train, y_test = tts(X,y,random_state=17, test_size = 0.33)
#Random Forest
rfc = RFC(random_state = 146, n_estimators = 100, max_depth = 18)
rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_test)
print("RFC SCORE",rfc.score(X_train, y_train))
print(CM(y_test,y_pred))
print(classification_report(y_test,y_pred))


RFC SCORE 0.9993011182108626
[[1881   18    2   18]
 [  24  867   12   39]
 [   3   24  430    2]
 [  27    9    1 1577]]
              precision    recall  f1-score   support

           0       0.97      0.98      0.98      1919
           1       0.94      0.92      0.93       942
           2       0.97      0.94      0.95       459
           3       0.96      0.98      0.97      1614

    accuracy                           0.96      4934
   macro avg       0.96      0.95      0.96      4934
weighted avg       0.96      0.96      0.96      4934



In [ ]:
def show_confusion_matrix(confusion_matrix):
  hmap = sns.heatmap(confusion_matrix, annot=True, fmt="d", cmap="Blues")
  hmap.yaxis.set_ticklabels(hmap.yaxis.get_ticklabels(), rotation=0, ha='right')
  hmap.xaxis.set_ticklabels(hmap.xaxis.get_ticklabels(), rotation=30, ha='right')
  plt.ylabel('True Category')
  plt.xlabel('Predicted Category')

In [ ]:
#DTC

In [ ]:
#KNN